In [1]:
import pandas as pd
from sqlalchemy import create_engine
import geopandas as gpd
import folium
from folium import FeatureGroup, LayerControl, Map, Marker
import pymongo
import numpy as np

### Checking new data with json file

In [2]:
#checking json file to see if it is usable, looks no much use to our purposes, so we only use it as refrence
json_file = "Resources/package_show.json"
js_df = pd.read_json(json_file)
js_df.head()

,help,success,result
access_level,https://catalogue.data.wa.gov.au/api/3/action/...,True,open
author,https://catalogue.data.wa.gov.au/api/3/action/...,True,Mainroads
author_email,https://catalogue.data.wa.gov.au/api/3/action/...,True,gis@mainroads.wa.gov.au
creator_user_id,https://catalogue.data.wa.gov.au/api/3/action/...,True,c9ab9aaa-8736-4a6c-ab12-5411196ad88f
data_homepage,https://catalogue.data.wa.gov.au/api/3/action/...,True,https://portal-mainroads.opendata.arcgis.com/d...


### Store geojson data into a DataFrame

In [3]:
#read in geojson file and convert into df
fname = "Resources/Roadworks.geojson"

gjs_df = gpd.read_file(fname)
gjs_df.head()

,OBJECTID,Id,DateStarted,EstimatedCompletionDate,WorkType,Description,Suburb,Road,TrafficImpact,Region,EntryDate,geometry
0,175229,45919,31/12/2019 06:00:00,31/12/2021 18:00:00,Maintenance,"Riverside Dr near William St, Perth City - Mai...",Perth City,Riverside Dr,Lane closures and speed reductions,Metro,17/12/2019 13:03:25,POINT (115.85503 -31.95795)
1,176544,52430,24/02/2020 22:00:00,11/03/2021 05:00:00,Resurfacing,"Morley Dr at Alexander Dr, Dianella - Resurfa...",Dianella,Morley Dr,Various lane closures,Metro,14/02/2020 14:31:57,POINT (115.87191 -31.88979)
2,180187,53485,25/03/2020 00:00:00,25/03/2021 00:00:00,Maintenance,Tuart Dr Layman Rd Wonnerup - Maintenance,Wonnerup,LOCAL ROAD,Road closed until further notice\r\nDetour in ...,South West,03/04/2020 18:59:48,POINT (115.43171 -33.63915)
3,180220,53554,12/04/2020 18:00:00,04/04/2021 06:00:00,Maintenance,"Karrinyup Rd at Jeanes Rd, Karrinyup - Mainte...",Karrinyup,Karrinyup Rd,Eastbound left lane closure first Sunday of ev...,Metro,07/04/2020 17:12:50,POINT (115.77667 -31.87775)
4,189131,54724,10/06/2020 10:00:00,31/05/2021 11:00:00,Upgrades,Marmion St eastbound right turn to Stirling Hw...,,LOCAL ROAD,"Closed, Follow signed detour",Metro,12/06/2020 13:56:13,POINT (115.76445 -32.04734)


In [4]:
#take consider only the metro part
megjs_df = gjs_df.loc[gjs_df["Region"] == "Metro", :]
megjs_df.head()

,OBJECTID,Id,DateStarted,EstimatedCompletionDate,WorkType,Description,Suburb,Road,TrafficImpact,Region,EntryDate,geometry
0,175229,45919,31/12/2019 06:00:00,31/12/2021 18:00:00,Maintenance,"Riverside Dr near William St, Perth City - Mai...",Perth City,Riverside Dr,Lane closures and speed reductions,Metro,17/12/2019 13:03:25,POINT (115.85503 -31.95795)
1,176544,52430,24/02/2020 22:00:00,11/03/2021 05:00:00,Resurfacing,"Morley Dr at Alexander Dr, Dianella - Resurfa...",Dianella,Morley Dr,Various lane closures,Metro,14/02/2020 14:31:57,POINT (115.87191 -31.88979)
3,180220,53554,12/04/2020 18:00:00,04/04/2021 06:00:00,Maintenance,"Karrinyup Rd at Jeanes Rd, Karrinyup - Mainte...",Karrinyup,Karrinyup Rd,Eastbound left lane closure first Sunday of ev...,Metro,07/04/2020 17:12:50,POINT (115.77667 -31.87775)
4,189131,54724,10/06/2020 10:00:00,31/05/2021 11:00:00,Upgrades,Marmion St eastbound right turn to Stirling Hw...,,LOCAL ROAD,"Closed, Follow signed detour",Metro,12/06/2020 13:56:13,POINT (115.76445 -32.04734)
5,191393,55389,12/07/2020 07:00:00,11/07/2021 11:00:00,Maintenance,"Canning Hwy at Kwinana Fwy, Como - Maintenanc...",Como,Canning Hwy,Lane closure and speed reductions,Metro,08/07/2020 09:32:40,POINT (115.85665 -32.00903)


In [5]:
#simplify the original df
metro_df = megjs_df[["Id", "DateStarted", "EstimatedCompletionDate", "WorkType", "Description", "TrafficImpact", "geometry"]].copy()
metro_df.head()

,Id,DateStarted,EstimatedCompletionDate,WorkType,Description,TrafficImpact,geometry
0,45919,31/12/2019 06:00:00,31/12/2021 18:00:00,Maintenance,"Riverside Dr near William St, Perth City - Mai...",Lane closures and speed reductions,POINT (115.85503 -31.95795)
1,52430,24/02/2020 22:00:00,11/03/2021 05:00:00,Resurfacing,"Morley Dr at Alexander Dr, Dianella - Resurfa...",Various lane closures,POINT (115.87191 -31.88979)
3,53554,12/04/2020 18:00:00,04/04/2021 06:00:00,Maintenance,"Karrinyup Rd at Jeanes Rd, Karrinyup - Mainte...",Eastbound left lane closure first Sunday of ev...,POINT (115.77667 -31.87775)
4,54724,10/06/2020 10:00:00,31/05/2021 11:00:00,Upgrades,Marmion St eastbound right turn to Stirling Hw...,"Closed, Follow signed detour",POINT (115.76445 -32.04734)
5,55389,12/07/2020 07:00:00,11/07/2021 11:00:00,Maintenance,"Canning Hwy at Kwinana Fwy, Como - Maintenanc...",Lane closure and speed reductions,POINT (115.85665 -32.00903)


In [6]:
#rename the column name "EstimatedCompletionDate" into "FinishDate"
rmetro_df = metro_df.rename(columns = {"EstimatedCompletionDate":"FinishDate", "DateStarted": "StartDate"})
rmetro_df.head()

,Id,StartDate,FinishDate,WorkType,Description,TrafficImpact,geometry
0,45919,31/12/2019 06:00:00,31/12/2021 18:00:00,Maintenance,"Riverside Dr near William St, Perth City - Mai...",Lane closures and speed reductions,POINT (115.85503 -31.95795)
1,52430,24/02/2020 22:00:00,11/03/2021 05:00:00,Resurfacing,"Morley Dr at Alexander Dr, Dianella - Resurfa...",Various lane closures,POINT (115.87191 -31.88979)
3,53554,12/04/2020 18:00:00,04/04/2021 06:00:00,Maintenance,"Karrinyup Rd at Jeanes Rd, Karrinyup - Mainte...",Eastbound left lane closure first Sunday of ev...,POINT (115.77667 -31.87775)
4,54724,10/06/2020 10:00:00,31/05/2021 11:00:00,Upgrades,Marmion St eastbound right turn to Stirling Hw...,"Closed, Follow signed detour",POINT (115.76445 -32.04734)
5,55389,12/07/2020 07:00:00,11/07/2021 11:00:00,Maintenance,"Canning Hwy at Kwinana Fwy, Como - Maintenanc...",Lane closure and speed reductions,POINT (115.85665 -32.00903)


In [7]:
#add a empty map column to be filled by maps
rmetro_df["map"] = ""
rmetro_df.reset_index(drop = True, inplace = True)
#rmetro_df.head()

In [8]:
#build coordinates list 
geo_df_list = [[point.xy[1][0], point.xy[0][0]] for point in rmetro_df.geometry ]

In [9]:
#build job id list
job_list = [j for j in rmetro_df.Id]
#job_list

In [10]:
#build road map for each roadwork, hovered show with job_id, click shows WorkType 
#For each job in consideration, its marker will be red and in the centre of the map 
maps = []

for j in range(len(geo_df_list)):
    feature_group = FeatureGroup(name= rmetro_df.iloc[j, 0])
    m = folium.Map(location = geo_df_list[j], zoom_start=12)

    for i in range(len(geo_df_list)):

        if rmetro_df.iloc[i, 0] == job_list[j]:
            tooltip = rmetro_df.iloc[j, 0]
            Marker(
            location = geo_df_list[j],
            popup = str(rmetro_df.iloc[j, 3]),
            icon=folium.Icon(color="red"), tooltip=tooltip
            ).add_to(feature_group)
        else:
            tooltip = rmetro_df.iloc[i, 0]
            location = geo_df_list[i]
            Marker(
            location, popup= str(rmetro_df.iloc[i, 3]), tooltip=tooltip
            ).add_to(feature_group)
            
    feature_group.add_to(m)    
    maps.append(m)         

In [11]:
#save the the representative map
maps[0].save("Perth_metro_roadwork.html")

In [12]:
maps[0]

In [13]:
#insert maps into rmetro_df.map
rmetro_df["map"] = maps
rmetro_df.head()

,Id,StartDate,FinishDate,WorkType,Description,TrafficImpact,geometry,map
0,45919,31/12/2019 06:00:00,31/12/2021 18:00:00,Maintenance,"Riverside Dr near William St, Perth City - Mai...",Lane closures and speed reductions,POINT (115.85503 -31.95795),<folium.folium.Map object at 0x0000018AA6CC6A20>
1,52430,24/02/2020 22:00:00,11/03/2021 05:00:00,Resurfacing,"Morley Dr at Alexander Dr, Dianella - Resurfa...",Various lane closures,POINT (115.87191 -31.88979),<folium.folium.Map object at 0x0000018AA6D032B0>
2,53554,12/04/2020 18:00:00,04/04/2021 06:00:00,Maintenance,"Karrinyup Rd at Jeanes Rd, Karrinyup - Mainte...",Eastbound left lane closure first Sunday of ev...,POINT (115.77667 -31.87775),<folium.folium.Map object at 0x0000018AA6D93A90>
3,54724,10/06/2020 10:00:00,31/05/2021 11:00:00,Upgrades,Marmion St eastbound right turn to Stirling Hw...,"Closed, Follow signed detour",POINT (115.76445 -32.04734),<folium.folium.Map object at 0x0000018AA6DCBA20>
4,55389,12/07/2020 07:00:00,11/07/2021 11:00:00,Maintenance,"Canning Hwy at Kwinana Fwy, Como - Maintenanc...",Lane closure and speed reductions,POINT (115.85665 -32.00903),<folium.folium.Map object at 0x0000018AA6E029B0>


In [14]:
type(maps[1])

folium.folium.Map

In [15]:
#To build MongoDB and store the DATA
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

db = client.etlDB
rworks = db.roadworks

In [23]:
db.mars.delete_many({})

In [21]:
#build general info field and insert into MongoDB
rwork = {}
gov_link = "https://catalogue.data.wa.gov.au/dataset/mrwa-roadworks"
geojson_link = "https://portal-mainroads.opendata.arcgis.com/datasets/f8df3952b47a4a73aac2df74a84a5b65_1.geojson"
csv_link = "https://portal-mainroads.opendata.arcgis.com/datasets/f8df3952b47a4a73aac2df74a84a5b65_1.csv" 
general_map = "Perth_metro_roadwok.html"
thanks = "Thank you for visit, enjoy, best regards from team Quokka"
rwork["source"] = gov_link
rwork["gjson"] = geojson_link
rwork["csv"] = csv_link
rwork["general_map"] = general_map
rwork["Thanks"] = thanks

db.rworks.insert_one(rwork)

In [22]:
#load the whole data into MongoDB
#However mongoDB doesn't accept folium map, I give it up for the maps' inserting.

for i in range(len(job_list)):
    bag = {}
    bag['Id'] = int(rmetro_df.iloc[i, 0])
    bag['StartDate'] = rmetro_df.iloc[i, 1]
    bag['FinishDate'] = rmetro_df.iloc[i, 2]
    bag['WorkType'] = rmetro_df.iloc[i, 3]
    bag['Description'] = rmetro_df.iloc[i, 4]
    bag['TrafficImpact'] = rmetro_df.iloc[i, 5]
    #bag['geometry'] = rmetro_df.iloc[i, 6]
    bag['map'] = rmetro_df.iloc[i, 7]
    
    db.rworks.insert_one(bag)

In [54]:
#db.mars.delete_many({})
n = db.rworks.count_documents({}) 
print (n)       

37


In [57]:
roadworks = db.rworks.find()
for r in roadworks:
    print(r)

{'_id': ObjectId('604acc6752696ec426c17dd0'), 'source': 'https://catalogue.data.wa.gov.au/dataset/mrwa-roadworks', 'gjson': 'https://portal-mainroads.opendata.arcgis.com/datasets/f8df3952b47a4a73aac2df74a84a5b65_1.geojson', 'csv': 'https://portal-mainroads.opendata.arcgis.com/datasets/f8df3952b47a4a73aac2df74a84a5b65_1.csv', 'Thanks': 'Thank you for visit, enjoy, best regards from team Quokka'}
{'_id': ObjectId('604acddc52696ec426c17dd1'), 'source': 'https://catalogue.data.wa.gov.au/dataset/mrwa-roadworks', 'gjson': 'https://portal-mainroads.opendata.arcgis.com/datasets/f8df3952b47a4a73aac2df74a84a5b65_1.geojson', 'csv': 'https://portal-mainroads.opendata.arcgis.com/datasets/f8df3952b47a4a73aac2df74a84a5b65_1.csv', 'general_map': 'Perth_metro_roadwok.html', 'Thanks': 'Thank you for visit, enjoy, best regards from team Quokka'}
{'_id': ObjectId('604ad2d652696ec426c17dda'), 'source': 'https://catalogue.data.wa.gov.au/dataset/mrwa-roadworks', 'gjson': 'https://portal-mainroads.opendata.ar